In [1]:
# !pip install transformers accelerate
# from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import Dataset
# %pip install evaluate
import evaluate
from sklearn.model_selection import train_test_split
# from ipywidgets import FloatProgress
import csv
from optuna import Trial
from typing import Dict, Union, Any
import os
import sys

# notebook_login()

csv.field_size_limit(500 * 1024 * 1024)
CUDA_LAUNCH_BLOCKING=1
# get pwd
notebook_path = os.path.abspath('')

# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'CommitFit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        break

if commit_fit_path is None:
    raise ValueError("Path containing 'commitFit' not found in notebook path.")

# Add commitFit directory to Python path, so we can import moudule from commitfit folder directly
if commit_fit_path not in sys.path:
    sys.path.append(commit_fit_path)

from commitfit import CommitFitModel, CommitFitTrainer 
from commitfit import get_templated_dataset,sample_dataset


root
autodl-tmp
CommitFit


In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'A':0, 'C':1, 'P':2}
df = df.rename(columns={'maintenance_type':'label','msgs':'text','commit_diff':'diffs'})
df = df.replace({"label": label2id})
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

/tmp/ipykernel_1376/1444002828.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,language,commit,commit_url,label,user,repo,feature,diffs,text
0,C++,be937a3290223f926fe50684f1344569a573ed4b,https://github.com/tensorflow/tensorflow/commi...,1,tensorflow,tensorflow,"[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 6, ...",diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...
1,C++,c800d2e36954edddcb83aa1df7f623f2780c7780,https://github.com/tensorflow/tensorflow/commi...,0,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...
2,C++,469e56eeff17eb857c95e935ca7b49723c43470e,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...
3,C++,24f5d302c0906f3377f4d6bcc2735d943ce13e3d,https://github.com/tensorflow/tensorflow/commi...,0,tensorflow,tensorflow,"[3, 37, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,..."
4,C++,1effdb7b959503596b07f7f3e74618ab63f9e5e5,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[1, 0, 12, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...
...,...,...,...,...,...,...,...,...,...
1576,Java,e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,,1,kiegroup,drools,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...
1577,Java,75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,,1,kiegroup,drools,"[2, 6, 2, 3, 0, 0, 0, 1, 7, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...
1578,Java,6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,,1,kiegroup,drools,"[1, 1, 5, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...
1579,Java,48701edad8513b27acec7216581e64637157c86a,,0,kiegroup,drools,"[2, 24, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,DROOLS-515 Kie-Camel is broken after Camel Upd...


In [3]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [4]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [5]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['Corrective','Adaptive','Perfective'], sample_size=8)

In [6]:
train_code_change = list(train['diffs'].astype(str))
test_code_change = list(test['diffs'].astype(str))

In [7]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['language', 'commit', 'commit_url', 'label', 'user', 'repo', 'feature', 'diffs', 'text', '__index_level_0__'],
    num_rows: 1130
})

In [8]:
len(train)

1106

In [9]:
train['label'].value_counts()

label
2    691
1    242
0    173
Name: count, dtype: int64

In [10]:
test['label'].value_counts()

label
2    155
1     51
0     32
Name: count, dtype: int64

In [11]:
len(train_dataset)

1130

In [12]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='macro')
    recall_score = metrics.recall_score(y_test,y_pred,average='macro')
    f1_score = metrics.f1_score(y_test,y_pred,average='macro')
    
    # 混淆矩阵
    cm = metrics.confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:\n", cm)
    print(metrics.classification_report(y_test, y_pred, digits=4))
    # return {"classification_report": classification_report}
    return {"classification_report":metrics.classification_report(y_test, y_pred, digits=4),"Confusion Matrix":cm,"precision": round(precision_score, 4),"recall": round(recall_score, 4), "f1":round(f1_score, 4), "accuracy": round(accuracy_score, 4)}

In [13]:
train_dataset

Dataset({
    features: ['language', 'commit', 'commit_url', 'label', 'user', 'repo', 'feature', 'diffs', 'text', '__index_level_0__'],
    num_rows: 1130
})

In [14]:
model_id = "../../sentence-transformers/paraphrase-mpnet-base-v2"

In [15]:
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True),
        # "num_epochs": trial.suggest_int("num_epochs", 1, 3),
        # "batch_size": trial.suggest_categorical("batch_size", [8, 12, 16]),
        # "seed": trial.suggest_int("seed", 1, 40),
        # "num_iterations": trial.suggest_int("num_iterations", 10, 20),
        # "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }
def model_init(params: Dict[str, Any]) -> CommitFitModel:
    params = params or {}
    # learning_rate = params.get("learning_rate")
    # num_iterations = params.get("num_iterations", 20)
    # solver = params.get("solver", "liblinear")
    # params = {
    #     "head_params": {
    #         # "max_iter": num_iterations,
    #         # "solver": solver,
    #     }
    # }
    return CommitFitModel.from_pretrained(model_id, **params)
def my_compute_objective(metrics):
    print('+++++++++++',metrics)
    return  metrics['accuracy']

In [16]:
trainer = CommitFitTrainer(
    train_dataset=train_dataset,
    train_code_change = train_code_change,
    test_code_change = test_code_change,
    eval_dataset=test_dataset,
    model_init=model_init,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, compute_objective=my_compute_objective, n_trials=10)

model_head.pkl not found in /root/autodl-tmp/CommitFit/sentence-transformers/paraphrase-mpnet-base-v2, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
[I 2025-08-31 13:44:37,330] A new study created in memory with name: no-name-c9e3b702-ba14-4261-a14e-e9ba0641db91
Trial: {'learning_rate': 5.988796181260846e-06}
model_head.pkl not found in /root/autodl-tmp/CommitFit/sentence-transformers/paraphrase-mpnet-base-v2, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 45200
  Num epochs = 1
  Total optimization steps = 2825
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2825 [00:00<?, ?it/s]

[W 2025-08-31 13:45:16,455] Trial 0 failed with parameters: {'learning_rate': 5.988796181260846e-06} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/root/autodl-tmp/CommitFit/commitfit/integrations.py", line 27, in _objective
    trainer.train(trial=trial)
  File "/root/autodl-tmp/CommitFit/commitfit/trainer.py", line 404, in train
    self.model.model_body.fit(
  File "/root/miniconda3/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py", line 705, in fit
    labels = labels.to(self._target_device)
KeyboardInterrupt
[W 2025-08-31 13:45:16,457] Trial 0 failed with value None.


KeyboardInterrupt: 

In [ ]:
best_run

In [ ]:
best_run.hyperparameters

In [ ]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

In [ ]:
fewshot_metrics = trainer.evaluate()
fewshot_metrics

In [ ]:
#发送多种类型的邮件
from email.mime.multipart import MIMEMultipart
import smtplib

from email.mime.text import MIMEText
msg_from = '915803745@qq.com'  # 发送方邮箱
passwd = 'vcuosuurrgkfbdai'   #就是上面的授权码
 
# to= ['g.zhang@gotion.com', 'j.tong@gotion.com'] #接受方邮箱
to= ['j.tong@gotion.com'] #接受方邮箱
#设置邮件内容
#MIMEMultipart类可以放任何内容
msg = MIMEMultipart()
conntent=f"{fewshot_metrics}"
#把内容加进去
msg.attach(MIMEText(conntent,'plain','utf-8'))
 
#设置邮件主题
msg['Subject']="强化学习模型训练完毕"
 
#发送方信息
msg['From']=msg_from
 
#开始发送
 
#通过SSL方式发送，服务器地址和端口
s = smtplib.SMTP_SSL("smtp.qq.com", 465)
# 登录邮箱
s.login(msg_from, passwd)
#开始发送
s.sendmail(msg_from,to,msg.as_string())
print("强化学习模型训练完毕")